In [0]:
pip install mlflow

In [0]:
import mlflow
import mlflow.sklearn
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_diabetes
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import f1_score
from sklearn.metrics import mean_absolute_error
from mlflow.tracking import MlflowClient
client = MlflowClient()

#### Data Import Step from dataset

In [0]:


db = load_diabetes()
X = db.data
y = db.target
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [0]:
X.shape, y.shape

## MLflow activity start 

In [0]:
mlflow.sklearn.autolog()

###### please set ==>mlflow.set_experiment("/Users/[Databrick-username]/[Expriment-name]")  

In [0]:
name_expriment = "Diabatics1"
Databrick_username = <Databricks-Username>
model_name = name_expriment
mlflow.create_experiment(f"/Users/{Databrick_username}/{name_expriment}")

Here we have set expriment name along with that we are collecting run_id in real time environment to automate the below process of model registring and keeping version of it.

For now, If you change the Hyper-parameter(n_estimators,max_depth,max_features) the model will auto register along with that version will be auto maintain to stagging.


we have use client.get_latest_versions(name=name_expriment) to get latest versions of model registed.

In [0]:
mlflow.set_experiment(f"/Users/{Databrick_username}/{name_expriment}")  
run_id = ''
with mlflow.start_run() as run:
  
  # Set the model parameters. 
  n_estimators = 200
  max_depth = 13
  max_features = 4
  
  # Create and train model.
  rf = RandomForestRegressor(n_estimators = n_estimators, max_depth = max_depth, max_features = max_features)
  rf.fit(X_train, y_train)
  predictions = rf.predict(X_test)
  err = mean_absolute_error(y_test, predictions)
  mlflow.log_metric("Score", err)
  run_id = run.info.run_id


#run_id = runs[0].info.run_id
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name=model_name)


latest_versions = client.get_latest_versions(name=model_name)
model_version = int(latest_versions[0].version)
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,  
)

In [0]:
model_name = name_expriment
latest_versions = client.get_latest_versions(name=model_name, stages=['Staging'])

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

In below cell, we have code for putting latest version to production

In [0]:
model_version = int(latest_versions[0].version)
new_stage = "Production"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=True
)

In [0]:
model_name = name_expriment
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

In [0]:
run_id = mlflow.search_runs(filter_string="run_name='bold-sow-291'")['run_id']


run_id